In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('mel_dataset.csv')

In [3]:
feature_cols = [col for col in df.columns if col.startswith('mel_')]
X = df[feature_cols].values
y = df['Ragas'].values
print("Shape of X before reshape:", X.shape)
print("Shape of y before encode:", y.shape)

Shape of X before reshape: (170, 165376)
Shape of y before encode: (170,)


In [5]:
# 2. Preprocessing
# Reshape the Mel Spectrograms into a 2D format
# The original shape is (samples, features). We need to reshape it into
# (samples, height, width, channels).
#side_length = int(np.sqrt(X.shape[1]))
#if side_length * side_length != X.shape[1]:
    #raise ValueError("The number of features is not a perfect square. Cannot reshape to a square 2D image.")

#X_reshaped = X.reshape(X.shape[0], side_length, side_length, 1)
X_reshaped = X.reshape(170, 16, 10336, 1)

In [6]:
# Encode the labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
num_classes = len(np.unique(y_encoded))
y_categorical = to_categorical(y_encoded, num_classes=num_classes)
print("Shape of y_categorical before split:", y_categorical.shape)

Shape of y_categorical before split: (170, 9)


In [7]:
print(num_classes)

9


In [8]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_reshaped, y_categorical, test_size=0.2, random_state=42)

In [23]:
#2D CNN Model
def create_cnn_model(input_shape, num_classes):
    model = Sequential([
        # Convolutional Block 1
        #Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        #MaxPooling2D((2, 2)),

        # Convolutional Block 2
        Conv2D(64, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),

        # Convolutional Block 3
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),

        # Flatten and Dense Layers
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5), # Dropout for regularization
        Dense(num_classes, activation='softmax') # Output layer
    ])
    return model

In [24]:
input_shape = (16, 10336, 1)
model = create_cnn_model(input_shape, num_classes=9)

In [25]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [26]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 14, 10334, 64)  │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 7, 5167, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 5, 5165, 128)   │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 2, 2582, 128)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 660992)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │   169,214,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 9)              │         2,313 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 169,291,017 (645.79 MB)

 Trainable params: 169,291,017 (645.79 MB)

 Non-trainable params: 0 (0.00 B)

In [27]:
history = model.fit(X_train, y_train,epochs=20, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 83s 14s/step - accuracy: 0.2031 - loss: 972.0398 - val_accuracy: 0.1765 - val_loss: 61.6610
Epoch 2/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 73s 14s/step - accuracy: 0.2056 - loss: 89.6582 - val_accuracy: 0.4118 - val_loss: 1.6752
Epoch 3/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 85s 14s/step - accuracy: 0.3945 - loss: 13.7315 - val_accuracy: 0.3824 - val_loss: 5.8602
Epoch 4/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 81s 14s/step - accuracy: 0.4919 - loss: 10.5555 - val_accuracy: 0.3824 - val_loss: 1.8744
Epoch 5/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 70s 14s/step - accuracy: 0.5825 - loss: 3.0153 - val_accuracy: 0.4412 - val_loss: 1.6877
Epoch 6/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 82s 14s/step - accuracy: 0.7035 - loss: 1.8103 - val_accuracy: 0.2941 - val_loss: 4.4122
Epoch 7/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 83s 14s/step - accuracy: 0.6256 - loss: 2.9217 - val_accuracy: 0.2647 - val_loss: 1.9843
Epoch 8/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 87s 15s/step - accuracy: 0.4272 - loss: 2.1516 - val_accuracy: 0.2647 - val_loss: 1

In [28]:
# 5. Evaluate the Model
loss, accuracy = model.evaluate(X_val, y_val, verbose=0)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")

Validation Loss: 1.4314
Validation Accuracy: 0.4706
